In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from scipy.sparse import hstack
from sklearn.metrics import roc_auc_score, accuracy_score, log_loss
from tqdm import tqdm_notebook, tqdm
from scipy import stats
import numpy as np
import pandas as pd
import warnings
from matplotlib import pyplot as plt
%matplotlib inline
from sklearn.metrics import mean_squared_error, mean_squared_log_error
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate
from sklearn import model_selection
import xgboost as xgb
import lightgbm as lgb
from sklearn import metrics
from sklearn.cluster import KMeans

from tqdm.notebook import tqdm
from scipy.optimize import minimize
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import RandomForestRegressor
warnings.filterwarnings('ignore')

In [ ]:
train = pd.read_csv('/kaggle/input/tabular-playground-series-jan-2021/train.csv')
test = pd.read_csv('/kaggle/input/tabular-playground-series-jan-2021/test.csv')
sub = pd.read_csv('/kaggle/input/tabular-playground-series-jan-2021/sample_submission.csv')

In [ ]:
train.head()

In [ ]:
train.shape

In [ ]:
train['target'].max(), train['target'].min(), 

In [ ]:
train['target'].max()

In [ ]:
y = train['target']
train.drop('target', axis=1, inplace=True)

In [ ]:
train.head()

In [ ]:
test_id = test['id']
train.drop('id', axis=1, inplace=True)
test.drop('id', axis=1, inplace=True)

In [ ]:
train['target'] = y

In [ ]:
train = train.sort_values(by='target').reset_index(drop=True)

In [ ]:
data=pd.concat([train,test],sort=False).reset_index(drop=True)
data.columns.tolist()

# selecting top 5 features to aggregate 

In [ ]:
features= ['cont13',
 'cont10','cont6', 'cont3', 'cont2']

In [ ]:
def aggregate_features(x,col_name):
    x["max_"+col_name]=x[col_name].apply(np.max)
    x["min_"+col_name]=x[col_name].apply(np.min)
    x["mean_"+col_name]=x[col_name].apply(np.mean)
    x["std_"+col_name]=x[col_name].apply(np.std)
    x["median_"+col_name]=x[col_name].apply(np.median)
    return x  

In [ ]:
for col_name in tqdm(features):
    data=aggregate_features(data,col_name)

In [ ]:
train=data[data.target.notnull()].reset_index(drop=True)
test=data[data.target.isna()].reset_index(drop=True)

In [ ]:
test = test.drop('target', axis=1)

In [ ]:
train.shape, test.shape

In [ ]:
X = train.drop('target', axis=1)
y = train['target']

In [ ]:
# params from this kernel https://www.kaggle.com/kailex/tabular-playground

params={'random_state': 33,'n_estimators':5000,
 'min_data_per_group': 5,
 'boosting_type': 'gbdt',
 'num_leaves': 256,#256
 'max_dept': -1,
 'learning_rate': 0.02,#0.02
 'subsample_for_bin': 200000,
 'lambda_l1': 1.074622455507616e-05,
 'lambda_l2': 2.0521330798729704e-06,
 'n_jobs': -1,
 'cat_smooth': 1.0,
 'silent': True,
 'importance_type': 'split',
 'metric': 'rmse',
 'feature_pre_filter': False,
 'bagging_fraction': 0.8206341150202605,
 'min_data_in_leaf': 100,
 'min_sum_hessian_in_leaf': 0.001,
 'bagging_freq': 6,
 'feature_fraction': 0.35,
 'min_gain_to_split': 0.0,
 'min_child_samples': 20}

In [ ]:
from sklearn.metrics import accuracy_score,confusion_matrix,log_loss
from sklearn.model_selection import GroupKFold, RepeatedKFold
errcb=[]
y_pred_totcb=[]
fold= KFold(n_splits=10, shuffle=True, random_state=1)
i=1
for train_index, test_index in fold.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    m = LGBMRegressor(**params)
    m.fit(X_train,y_train,eval_set=[(X_train,y_train),(X_test, y_test)],verbose=100, early_stopping_rounds=100)#,verbose=100)
    preds=m.predict(X_test)
    print("err: ",np.sqrt(mean_squared_error(y_test,preds)))
    errcb.append(np.sqrt(mean_squared_error(y_test,preds)))
    p = m.predict(test)
    y_pred_totcb.append(p)

In [ ]:
np.mean(errcb)

In [ ]:
fea_imp = pd.DataFrame({'imp':m.feature_importances_, 'col': X.columns})
fea_imp = fea_imp.sort_values(['imp', 'col'], ascending=[True, False]).iloc[-30:]
_ = fea_imp.plot(kind='barh', x='col', y='imp', figsize=(20, 10))
plt.savefig('rf.png')    

In [ ]:
d = {"id": test_id, 'target': np.mean(y_pred_totcb, 0)}
pred = pd.DataFrame(data=d)
subs = pred[["id", 'target']]

In [ ]:
subs.head()

In [ ]:
subs.to_csv('jan30.csv', index=False)